In [7]:
import configparser
import psycopg2
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import boto3
import json

In [14]:
# CONFIG
config = configparser.ConfigParser()
config.read_file(open('dwh_infra.cfg'))

# KEY                     = config.get('AWS','KEY')
# SECRET                  = config.get('AWS','SECRET')
# SESSION_TOKEN           = config.get('AWS','SESSION_TOKEN')

DB_NAME                 = config.get("CLUSTER","DB_NAME")
DB_USER                 = config.get("CLUSTER","DB_USER")
DB_PASSWORD             = config.get("CLUSTER","DB_PASSWORD")
DB_PORT                 = config.get("CLUSTER","DB_PORT")

IAM_ROLE_NAME            = config.get("IAM", "ROLE_NAME")

pd.DataFrame({"Param":
                  ["DB_NAME", "DB_USER", "DB_PASSWORD", "DB_PORT", "IAM_ROLE_NAME"],
              "Value":
                  [DB_NAME, DB_USER, DB_PASSWORD, DB_PORT, IAM_ROLE_NAME]
             })

,Param,Value
0,DB_NAME,dwh
1,DB_USER,dwhuser
2,DB_PASSWORD,Passw0rd
3,DB_PORT,5439
4,IAM_ROLE_NAME,musical-db-user-role


### Creating IAM User, EC2, S3 and Redshift

In [16]:
# import boto3

# ec2 = boto3.resource('ec2',
#                        region_name="us-west-2",
#                        aws_access_key_id=KEY,
#                        aws_secret_access_key=SECRET,
#                        aws_session_token=SESSION_TOKEN
#                     )

# s3 = boto3.resource('s3',
#                        region_name="us-west-2",
#                        aws_access_key_id=KEY,
#                        aws_secret_access_key=SECRET,
#                        aws_session_token=SESSION_TOKEN
#                    )

# iam = boto3.client('iam', aws_access_key_id=KEY,
#                      aws_secret_access_key=SECRET,
#                      region_name='us-west-2',
#                      aws_session_token=SESSION_TOKEN
#                   )

# redshift = boto3.client('redshift',
#                        region_name="us-west-2",
#                        aws_access_key_id=KEY,
#                        aws_secret_access_key=SECRET,
#                        aws_session_token=SESSION_TOKEN
#                        )

In [20]:

ec2 = boto3.resource('ec2',
                       region_name="us-west-2"
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2"
                   )

iam = boto3.client('iam', 
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2"
                       )

In [21]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
for obj in sampleDbBucket.objects.filter(Prefix="ssbgz"):
    print(obj)
# for obj in sampleDbBucket.objects.all():
#     print(obj)

s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/customer0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/dwdate.tbl.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0000_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0001_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0003_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0004_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0005_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0006_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0007_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='s

### Creating IAM Role for DB

In [22]:
print(SECRET)

8T3gYMF0WQtw4bm+fDCVz9taBn7Ji7vSEBs27di9


In [23]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)

print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::856840857391:role/musical-db-user-role


### Creating a Redshift Cluster

In [ ]:
# try:
#     response = redshift.create_cluster(        
#         #HW
#         ClusterType=DWH_CLUSTER_TYPE,
#         NodeType=DWH_NODE_TYPE,
#         NumberOfNodes=int(DWH_NUM_NODES),

#         #Identifiers & Credentials
#         DBName=DWH_DB,
#         ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
#         MasterUsername=DWH_DB_USER,
#         MasterUserPassword=DWH_DB_PASSWORD,
        
#         #Roles (for s3 access)
#         IamRoles=[roleArn]  
#     )
# except Exception as e:
#     print(e)

Describing its status

In [ ]:
# def prettyRedshiftProps(props):
#     pd.set_option('display.max_colwidth', -1)
#     keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
#     x = [(k, v) for k,v in props.items() if k in keysToShow]
#     return pd.DataFrame(data=x, columns=["Key", "Value"])

# myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
# prettyRedshiftProps(myClusterProps)